In [ ]:
# for x in dataset["input_ids"]:
#     print(tokenizer.decode(x))

# unique_sequences = set()
# for x in dataset["input_ids"]:
#     sequence = tokenizer.decode(x)
#     unique_sequences.add(sequence)

# print(f"Number of unique sequences: {len(unique_sequences)}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

tokens = ["A", "C", "D", "E", "F", "G", "H", "I", "K", "L", "M", "N", "P", "Q", "R", "S", "T", "V", "W", "Y"]
token2idx = dict(zip(tokens, range(len(tokens))))
idx2token = dict(zip(range(len(tokens)), tokens))

model.eval()

dataset_reloaded = load_from_disk("../tmp/data/pssm/pssm_dataset_0_only")
dataset_reloaded = dataset_reloaded.rename_column("pssm_features", "labels")
print(dataset_reloaded)


def plot_pssm_heatmap(probs, tile_extension="", figsize=(12, 8)):
    plt.figure(figsize=figsize)

    if torch.is_tensor(probs):
        probs = probs.cpu().numpy()

    sns.heatmap(probs, xticklabels=tokens, yticklabels=True, cmap="YlOrRd", vmin=0, vmax=1, cbar_kws={"label": "Probability"})

    plt.xlabel("Amino Acids")
    plt.ylabel("Sequence Position")
    plt.title(f"PSSM Heatmap {tile_extension}")
    plt.tight_layout()
    plt.show()


with torch.no_grad():
    index = 385
    for x in range(index, index + 2):
        test_input_ids = torch.tensor([dataset_reloaded[x]["input_ids"]]).to(device)
        test_attention_mask = torch.tensor([dataset_reloaded[x]["attention_mask"]]).to(device)

        seq_len = (test_attention_mask[0] == 1).sum() - 1
        print(f"Sequence length: {seq_len}")

        outs = model.forward(
            input_ids=test_input_ids,
            attention_mask=test_attention_mask,
            return_dict=True,
        )

        pred_probs = outs.logits[0][:seq_len]
        target_probs = torch.tensor(dataset_reloaded[x]["labels"]).to(device)

        # display(pd.DataFrame(pred_probs.cpu()))
        plot_pssm_heatmap(pd.DataFrame(pred_probs.cpu()), tile_extension=f"Predicted {dataset_reloaded[x]['name']}")

        # display(pd.DataFrame(target_probs.cpu()))
        plot_pssm_heatmap(pd.DataFrame(target_probs.cpu()), tile_extension=f"Truth {dataset_reloaded[x]['name']}")

        kl_div_1 = torch.nn.functional.kl_div(torch.log(pred_probs + 1e-10), target_probs, reduction="batchmean")
        kl_div_2 = torch.nn.functional.kl_div(torch.log(target_probs + 1e-10), pred_probs, reduction="batchmean")
        print(f"KL Divergence 1: {kl_div_1:.4f}")
        print(f"KL Divergence 2: {kl_div_2:.4f}")
        print(f"KL Divergence Total: {kl_div_1 + kl_div_2:.4f}")
        print("-" * 100)

model.train()
print()